In [1]:
import pathlib
import warnings

import numpy as np
import pandas as pd
import plotly.express as px
from copairs.map import aggregate

warnings.filterwarnings("ignore")

In [2]:
# Directories
processed_data_dir = pathlib.Path("../data/processed/")
figures_dir = pathlib.Path("../figures/").resolve()
figures_dir.mkdir(exist_ok=True)
sc_ap_scores_dir = (processed_data_dir / "mAP_scores").resolve()
agg_sc_ap_scores_dir = (processed_data_dir / "aggregate_mAPs").resolve()
agg_sc_ap_scores_dir.mkdir(exist_ok=True)

## Preparing the dataset


In [3]:
all_files = list(sc_ap_scores_dir.glob("*.csv"))
# get the files that contain the string class
class_files = [file for file in all_files if "class" in file.stem]
mAPs = []
for file in class_files:
    df = pd.read_csv(file)
    df["file"] = file.stem
    mAPs.append(df)
# single-cell mAP scores
mAPs = pd.concat(mAPs)
mAPs.head()

,Metadata_Well,Metadata_labels,average_precision,p_value,n_pos_pairs,n_total_pairs,shuffled,file
0,B02,Apoptosis,0.059487,0.5,7,153,features_shuffled,merged_sc_agg_ap_scores_class
1,D06,Apoptosis,0.640446,1.0,7,153,non-shuffled,merged_sc_agg_ap_scores_class
2,B02,Apoptosis,0.089383,1.0,7,153,phenotype_shuffled,merged_sc_agg_ap_scores_class
3,B04,Control,0.530483,1.0,80,153,features_shuffled,merged_sc_agg_ap_scores_class
4,B06,Control,0.603318,0.5,80,153,non-shuffled,merged_sc_agg_ap_scores_class


In [4]:
# Separating data frames: One by feature type (CP, DP, CP_DP)
# Additional split is performed using a shuffling approach:
# - feature_shuffled: feature values within the feature space are shuffled.
# - phenotype_shuffled: phenotypic labels are shuffled.

# grabbing all cp features (regular, feature shuffled and labeled shuffled)
reg_sc_mAPs = mAPs.loc[mAPs["shuffled"] == "non-shuffled"]
shuffled_feat_sc_mAPs = mAPs.loc[mAPs["shuffled"] == "features_shuffled"]
shuffled_pheno_sc_mAPs = mAPs.loc[mAPs["shuffled"] == "phenotype_shuffled"]

In [5]:
# Generating sampling_error df
# This table will be used to merge with the aggregate table to get the sampling error a specific category.
merged_sc_ap_scores_df = pd.concat(
    [
        reg_sc_mAPs,
        shuffled_feat_sc_mAPs,
        shuffled_pheno_sc_mAPs,
    ]
)

# grouping dataframe based on phenotype levels, feature and feature types
df_group = merged_sc_ap_scores_df.groupby(by=["Metadata_labels", "shuffled"])

# calculating sampling error
sampling_error_df = []
for name, df in df_group:
    pheno, shuffled_type = name

    # caclulating sampling error
    avg_percision = df["average_precision"].values
    sampling_error = np.std(avg_percision) / np.sqrt(len(avg_percision))

    sampling_error_df.append([pheno, shuffled_type, sampling_error])
cols = ["Metadata_labels", "shuffled", "sampling_error"]
sampling_error_df = pd.DataFrame(sampling_error_df, columns=cols)

# updating name:
sampling_error_df.loc[
    sampling_error_df["shuffled"] == "phenotype_shuffled"
] = "phenotypes_shuffled"

sampling_error_df.head()

,Metadata_labels,shuffled,sampling_error
0,Apoptosis,features_shuffled,0.004997
1,Apoptosis,non-shuffled,0.085021
2,phenotypes_shuffled,phenotypes_shuffled,phenotypes_shuffled
3,Control,features_shuffled,0.004709
4,Control,non-shuffled,0.011942


In [6]:
# aggregate single cells scores with cell UUID
data = tuple(merged_sc_ap_scores_df.groupby(by=["Metadata_labels"]))
columns = merged_sc_ap_scores_df.columns
agg_sc_ap_scores_df = []
for cell_id, df1 in data:
    for shuffle_type, df2 in df1.groupby(by="shuffled"):
        aggregated_ap_score = df2["average_precision"].mean()

        # select a single row since all the metadata is the same
        selected_row = df2.iloc[0]

        # update the average precision score of the single row
        selected_row["average_precision"] = aggregated_ap_score
        agg_sc_ap_scores_df.append(selected_row.values.tolist())

# saving into the results repo
agg_sc_ap_scores_df = pd.DataFrame(data=agg_sc_ap_scores_df, columns=columns)
agg_sc_ap_scores_df.to_csv(
    sc_ap_scores_dir / "merged_sc_agg_ap_scores_class.csv", index=False
)
agg_sc_ap_scores_df.head()

,Metadata_Well,Metadata_labels,average_precision,p_value,n_pos_pairs,n_total_pairs,shuffled,file
0,B02,Apoptosis,0.059487,0.5,7,153,features_shuffled,merged_sc_agg_ap_scores_class
1,D06,Apoptosis,0.640446,1.0,7,153,non-shuffled,merged_sc_agg_ap_scores_class
2,B02,Apoptosis,0.089383,1.0,7,153,phenotype_shuffled,merged_sc_agg_ap_scores_class
3,B04,Control,0.530483,1.0,80,153,features_shuffled,merged_sc_agg_ap_scores_class
4,B06,Control,0.603318,0.5,80,153,non-shuffled,merged_sc_agg_ap_scores_class


In [7]:
# Generating aggregate scores with a threshold p-value of 0.05
mAP_dfs = []
for name, df in tuple(agg_sc_ap_scores_df.groupby(by=["Metadata_labels", "shuffled"])):
    agg_df = aggregate(df, sameby=["Metadata_labels"], threshold=0.05)
    agg_df["Metadata_labels"] = name[0]
    agg_df["shuffled"] = name[1]
    mAP_dfs.append(agg_df)

mAP_dfs = pd.concat(mAP_dfs)
mAP_dfs.to_csv(agg_sc_ap_scores_dir / "mAP_scores_class.csv", index=False)
mAP_dfs.head()

,Metadata_labels,mean_average_precision,nlog10pvalue,q_value,nlog10qvalue,above_p_threshold,above_q_threshold,shuffled
0,Apoptosis,0.059487,0.30103,0.5,0.30103,False,False,features_shuffled
0,Apoptosis,0.640446,-0.00000,1.0,-0.00000,False,False,non-shuffled
0,Apoptosis,0.089383,-0.00000,1.0,-0.00000,False,False,phenotype_shuffled
0,Control,0.530483,-0.00000,1.0,-0.00000,False,False,features_shuffled
0,Control,0.603318,0.30103,0.5,0.30103,False,False,non-shuffled


## Forming bar plots


### Forming bar plots with CP Features


In [8]:
# selecting dataset to plot
agg_reg_sc_mAPs = mAP_dfs.loc[(mAP_dfs["shuffled"] == "non-shuffled")]
agg_shuffled_feat_sc_mAPs = mAP_dfs.loc[(mAP_dfs["shuffled"] == "features_shuffled")]
agg_shuffled_pheno_sc_mAPs = mAP_dfs.loc[(mAP_dfs["shuffled"] == "phenotype_shuffled")]

# phenotypes
df = (
    pd.concat(
        [
            agg_reg_sc_mAPs,
            agg_shuffled_feat_sc_mAPs,
            agg_shuffled_pheno_sc_mAPs,
        ]
    )
    .reset_index()
    .drop("index", axis=1)
)[["Metadata_labels", "mean_average_precision", "shuffled"]]


fig = px.bar(
    df,
    x="Metadata_labels",
    y="mean_average_precision",
    color="shuffled",
    barmode="group",
    title="Mean Average Precision for each Cell Death Phenotype",
    labels={
        "mean_average_precision": "Mean Average Precision",
        "Metadata_labels": "Cell Death Phenotypes",
    },
)

## Generating box plots of single cell ap scores

In [9]:
all_df = pd.concat(
    [
        reg_sc_mAPs,
        shuffled_feat_sc_mAPs,
        shuffled_pheno_sc_mAPs,
    ]
)

In [10]:
# Assuming all_cp_df, all_dp_df, and all_cp_dp_df are your DataFrames
categories_order = all_df["Metadata_labels"].unique()

# Create individual figures with the same category order
fig1 = px.box(
    all_df,
    x="Metadata_labels",
    y="average_precision",
    color="shuffled",
    title="Single Well Average Percision Scores",
    category_orders={"Metadata_labels": categories_order},
    labels={
        "average_precision": "Average Precision Scores",
        "Metadata_labels": "Cell Death Class",
    },
)